- Load necessary module

In [1]:
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import multinomial
from collections import OrderedDict

- Load model that defined in channel realization

In [2]:
mat = io.loadmat('temp.mat')
model = {}
model['H'] = mat['H']
model['Wopt'] = mat['Wopt']
model['Fopt'] = mat['Fopt']
model['Ns'] = 1 ## need to change nb of stream in channel realization, this case 1
model['Nt'] = mat['Nt'][0,0]
model['Nr'] = mat['Nr'][0,0]
model['Nc'] = mat['Nc'][0,0]
model['Nray'] = mat['Nray'][0,0]
model['realization'] = mat['realization'][0,0]
model['At'] = mat['At']
model['Ar'] = mat['Ar']
model['Nrf'] = 2

Load values of phase shifter included:
- nb_bit: number of bit
- nb_ps: number of phase shifter in the system
- nb_state: number of state in each phase shifter (2^nb_bit)

In [3]:
nb_bit = 2
nb_ps = model['Nt']*model['Nrf']
nb_ps
nb_state = 2**nb_bit

In [4]:
for key in model:
    print(key)

realization
Nray
Ns
At
Nt
Wopt
Nrf
Nr
Fopt
Nc
Ar
H


In [5]:
def phase_shifter(N):
    """
    Define phase shifter with N bits resolution
    
    Arguments:
    N -- number of bits resolutions
    
    Returns:
    y -- python dictionnary of phase shifter: index of state and value of phase
    """
    nb_state = 2**N
    w = np.exp(1j*2*np.pi/nb_state)
    ps = np.zeros(nb_state,dtype=complex)
    for i in range(nb_state):
        ps[i] = w**i
    return np.around(ps, decimals=2)

In [6]:
phase_value = phase_shifter(nb_bit)
phase_value

array([ 1.+0.j,  0.+1.j, -1.+0.j, -0.-1.j])

### Initialize phase shifter 
dimension is: NtxNrfxN_state

In [7]:
# Initialize phase shifter system
ps_system_proba = np.ones((model['Nt'],model['Nrf'],nb_state))*(1/nb_state)
# for i in range(nb_ps):
#     ps_system[i] = -11/

In [8]:
print(ps_system_proba.shape)
ps_system_proba

(64, 2, 4)


array([[[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25,

In [9]:
elements = [1, 2, 3, 4]
print(np.random.choice(elements, 10, p=ps_system_proba[0,0,:])) #phase_value
np.random.choice(phase_value, 1, p=ps_system_proba[0,0,:])

[3 3 1 4 3 3 4 3 4 2]


array([-0.-1.j])

In [10]:
ps_system_proba[0,0,:]

array([0.25, 0.25, 0.25, 0.25])

### Step 1: Randomly generate S candidate analog beamformers

In [ ]:
##########################
# S = 200
# F_rf = np.zeros((S,model['Nt'],model['Nrf']),dtype=complex)
# for i in range(S):
#     for j in range(model['Nt']):
#         for k in range(model['Nrf']):
#             F_rf[i,j,k] = np.random.choice(phase_value, 1, p=ps_system_proba[j,k,:])

In [12]:
def generate_candidat(S,model,phase_value):
    F_rf = np.zeros((S,model['Nt'],model['Nrf']),dtype=complex)
    for i in range(S):
        for j in range(model['Nt']):
            for k in range(model['Nrf']):
                F_rf[i,j,k] = np.random.choice(phase_value, 1, p=ps_system_proba[j,k,:])
    return F_rf

In [ ]:
F_rf = generate_candidat(S,model,phase_value)
F_rf.shape[0]

### Step 2: Compute S corresponding digital precoders Fbb based on the effective channel Heq = HFrf

In [13]:
### Test with only first channel from 1000 realizations
def compute_Fbb(F_rf,model):
    S = F_rf.shape[0]
    channel_eq = np.zeros((S,model['Nr'],model['Nrf']),dtype=complex)
    F_bb = np.zeros((S,model['Nrf'],model['Ns']),dtype=complex)
    channel_index = 0 ### need to adjust 
    for i in range(S):
        channel_eq[i,:,:] =  model['H'][:,:,channel_index]@F_rf[i,:,:]
        U, s, V = np.linalg.svd(channel_eq[i,:,:], full_matrices=True)
        V = V.conj().T
        F_bb[i,:,:] = V[0:model['Nt'],0:model['Ns']]
    return F_bb

In [ ]:
F_bb = compute_Fbb(F_rf,model)

In [ ]:
# channel_eq = np.zeros((S,model['Nr'],model['Nrf']),dtype=complex)
# F_bb = np.zeros((S,model['Nrf'],model['Ns']),dtype=complex)
# channel_index = 0
# for i in range(S):
#     channel_eq[i,:,:] =  model['H'][:,:,channel_index]@F_rf[i,:,:]
#     U, s, V = np.linalg.svd(channel_eq[i,:,:], full_matrices=True)
#     V = V.conj().T
#     F_bb[i,:,:] = V[0:model['Nt'],0:model['Ns']]

### Step 3: Calculate the achievable sum-rate

In [19]:
def optimal_combiner(model, channel_index = 0):
    Wopt = np.zeros((model['Nr'],model['Ns']),dtype=complex)
    U, s, V = np.linalg.svd(model['H'][:,:,channel_index], full_matrices=True)
    V = V.conj().T
    # Fopt[:,:,reali] = V[0:Nt,0:Ns]
    Wopt = U[0:model['Nr'],0:model['Ns']]
    return Wopt

In [ ]:
## Assume an optimize combining at Receiver

# Wopt = np.zeros((model['Nr'],model['Ns']),dtype=complex)
# U, s, V = np.linalg.svd(model['H'][:,:,channel_index], full_matrices=True)
# V = V.conj().T
# # Fopt[:,:,reali] = V[0:Nt,0:Ns]
# Wopt = U[0:model['Nr'],0:model['Ns']]
    

In [ ]:
# SNR_dB = np.arange(-35,10,5)
# SNR = 10**(SNR_dB/10)
# smax = SNR.shape[0]
SNR = 10**(10/10)
R = np.zeros((S))

for i in range(S):
    R[i] = np.log2(np.linalg.det(np.eye(model['Ns']))+SNR/model['Ns']*np.linalg.pinv(Wopt)@model['H'][:,:,channel_index]@F_rf[i,:,:]@F_bb[i,:,:]@F_bb[i,:,:].conj().T@F_rf[i,:,:].conj().T@model['H'][:,:,channel_index].conj().T@Wopt).real


In [ ]:
R

### Step 4: Sort R in a descend order

In [ ]:
nb_elite = 40

In [ ]:
R[49]

In [ ]:
R_sort_index = np.argsort(R)[::-1][:nb_elite]

In [ ]:
R_sort_index

In [ ]:
R_max_index = R_sort_index[0]
R_max_index

In [ ]:
print(R[R_sort_index])

In [ ]:
F_elites = F_rf[R_sort_index,:,:]

### Step 5: Select elites as Frf_1, Frf_2, ..., Frf_elite

In [ ]:
F_elites = F_rf[R_sort_index,:,:]

### Step 6: Update new pmf for each phase shifter

In [ ]:
F_elites.shape

In [ ]:
phase_value

In [ ]:
unique, counts = np.unique(F_elites[:,0,0], return_counts=True)
test = dict(zip(unique, counts))
print(test)
print(test[phase_value[3]])
 

In [ ]:
counts

In [ ]:
phase_value.shape[0]

In [ ]:
# stat_count = np.zeros(nb_state)
# for i in range(model['Nt']):
#     for j in range(model['Nrf']):
#         unique, counts = np.unique(F_elites[:,i,j], return_counts=True)
#         value_count = dict(zip(unique, counts))
#         for k in range(nb_state):
#             ps_system_proba[i,j,k] = value_count[phase_value[k]]/nb_elite
        
        

In [15]:
def update_pmf(F_elites,phase_value,nb_elite):
    nb_state = phase_value.shape[0]
    stat_count = np.zeros(nb_state)
    for i in range(model['Nt']):
        for j in range(model['Nrf']):
            unique, counts = np.unique(F_elites[:,i,j], return_counts=True)
            value_count = dict(zip(unique, counts))
            for k in range(nb_state):
                ps_system_proba[i,j,k] = value_count[phase_value[k]]/nb_elite
    return ps_system_proba

In [22]:
Wopt = optimal_combiner(model)
nb_elite = 40
S = 200
channel_index = 0
F_rf = generate_candidat(S,model,phase_value)
F_bb = compute_Fbb(F_rf,model)

SNR = 10**(10/10)
R = np.zeros((S))

for i in range(S):
    R[i] = np.log2(np.linalg.det(np.eye(model['Ns']))+SNR/model['Ns']*np.linalg.pinv(Wopt)@model['H'][:,:,channel_index]@F_rf[i,:,:]@F_bb[i,:,:]@F_bb[i,:,:].conj().T@F_rf[i,:,:].conj().T@model['H'][:,:,channel_index].conj().T@Wopt).real

R_sort_index = np.argsort(R)[::-1][:nb_elite]
R_max_index = R_sort_index[0]

F_elites = F_rf[R_sort_index,:,:]
ps_system_proba = update_pmf(F_elites,phase_value,nb_elite)


In [23]:
ps_system_proba

array([[[0.275, 0.375, 0.2  , 0.15 ],
        [0.35 , 0.125, 0.25 , 0.275]],

       [[0.275, 0.275, 0.175, 0.275],
        [0.375, 0.125, 0.225, 0.275]],

       [[0.325, 0.35 , 0.05 , 0.275],
        [0.2  , 0.3  , 0.175, 0.325]],

       [[0.175, 0.25 , 0.25 , 0.325],
        [0.35 , 0.3  , 0.225, 0.125]],

       [[0.1  , 0.35 , 0.3  , 0.25 ],
        [0.225, 0.375, 0.225, 0.175]],

       [[0.25 , 0.25 , 0.225, 0.275],
        [0.175, 0.275, 0.25 , 0.3  ]],

       [[0.25 , 0.275, 0.275, 0.2  ],
        [0.2  , 0.25 , 0.375, 0.175]],

       [[0.175, 0.15 , 0.275, 0.4  ],
        [0.2  , 0.25 , 0.275, 0.275]],

       [[0.3  , 0.35 , 0.2  , 0.15 ],
        [0.2  , 0.175, 0.35 , 0.275]],

       [[0.2  , 0.25 , 0.25 , 0.3  ],
        [0.2  , 0.2  , 0.3  , 0.3  ]],

       [[0.325, 0.2  , 0.225, 0.25 ],
        [0.3  , 0.275, 0.35 , 0.075]],

       [[0.225, 0.275, 0.15 , 0.35 ],
        [0.325, 0.125, 0.275, 0.275]],

       [[0.425, 0.175, 0.175, 0.225],
        [0.175, 0.15 , 0.2